In [28]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

team_ids = ['8586','9825','8456','8650','10252','10261','10204','10260','8654','9937','8455','8602','9826','9879','8668','10203','8678','8346','8191','8657',
             '9768','9772','9773','10264','8613','7844','6004','9764','9765','1074320','212821','158085','7842','9774','8338','1634','4531','7841',
             '8633','7732','9906','8634','8560','8315','8370','8603','10267','8306','8371','10205','8395','8302','8385','8661','9866','9910','7878','9865',
             '8517','10283','109705','9769','10272','9770','10273','9863','8287','10277','197815','9808','8702','7877','8514','9781','10276','9862','9767','1757',
             '8636','9885','8564','8524','9875','8535','8543','9857','8686','6504','9888','9891','9804','10233','7943','9876','8600','8534','8529','6480',
             '9831','9847','9829','8639','9837','8521','9830','9851','8592','8588','10249','9941','9746','9848','8689','8550','8311','9748',
             '8178','9823','10269','9789','178475','8226','9810','8358','8721','8406','9788','8697','94937','8262','8149','9911','8722','9905',
             '8640','10229','8611','10235','6433','8614','10218','9791','6413','10228','6422','8464','10219','4116','8593','8277','6601','9908',
             '8391','8595','8415','8113','10202','8071','9907','8410','9939','8414','8231','10240',
             '7978','8635','9991','9984','9987','8342','9988','9997','9985','6351','9986','9992','1773','8203','8571','10001',
             '8637','8695','1926','10188','4685','9752','10182','95749','2166','6265','9742','1931','2088','4678','95745','8622','4081','1933','106560','9750',
             '8197','9902','8463','8466','8659','8344','8667','8472','8411','8549','10194','8655','10003','8658','8427','9817','9850','10004','8401','8669','9796','8119','10172','10163',
             '7854','8372','8558','9869','8581','9867','10281','8696','8394','8474','10268','8670','4032','7876','8288','494050','8393','161771','189624','161750','96925','8554',
             '10076','10101','10083','10082','10094','161730','10077','10084','10227','10086','161727','10080','10092','10201','213534','89396','10089','10090','202757','10096','10087','10103','213596','10078','10079','10081','10088','10098',
             '9925','8548','9800','9860','8597','8485','10251','9927','8284','8649','8282','8467'] 
    #order eng1, pt1, esp1, bra1, ita1, fra1, ger1, ned1, den1, bel1, tur1, eng2, esp2, arg1, sco1
    # i made it by hand but it can be done by websscraping unfortunately wasnt able to do so
# Create an empty list to store all player information
all_players = []

# Iterate through each team ID
for team_id in team_ids:
    try:
        # Make the API request for each team
        url = f'https://www.fotmob.com/api/teams?id={team_id}&ccode3=PRT'
        response = requests.get(url, headers={'User-Agent': 'Chrome/115.0.0.0'})
        
        # Check if the response code is 200 (OK)
        if response.status_code == 200:
            team_info = response.json()

            # Extract the team name and league ID
            team_name = team_info['details']['name']
            league_id = team_info['overview']['table'][0]['data']['leagueId']

            # Extract player information for each role, excluding "coach"
            for role, player_list in team_info['squad']:
                if role == 'coach':
                    continue  # Skip coach information
                for player in player_list:
                    player_id = player['id']
                    player_name = player['name']
                    position = player.get('role', '')  # 'role' might not exist for some players
                    nation = player['ccode']
                    all_players.append([team_id, team_name, league_id, player_id, player_name, position, nation, role])
        else:
            print(f"Error for team ID {team_id}: Response code {response.status_code}")
    except Exception as e:
        print(f"Error for team ID {team_id}: {str(e)}")

# Create a Dataframe from all player data
df = pd.DataFrame(all_players, columns=['Team_ID', 'Team_Name', 'League_ID', 'Player_ID', 'Name', 'Position', 'Nation', 'Role'])

print(df)

Error for team ID 10088: 'NoneType' object is not iterable
Error for team ID 10098: 'NoneType' object is not iterable
     Team_ID          Team_Name  League_ID  Player_ID                Name  \
0       8586  Tottenham Hotspur         47      95336      Fraser Forster   
1       8586  Tottenham Hotspur         47     538501   Guglielmo Vicario   
2       8586  Tottenham Hotspur         47     795539      Alfie Whiteman   
3       8586  Tottenham Hotspur         47     862993      Brandon Austin   
4       8586  Tottenham Hotspur         47     163670        Ivan Perisic   
...      ...                ...        ...        ...                 ...   
8962    8467      St. Johnstone         64     172833         Nicky Clark   
8963    8467      St. Johnstone         64     314399    Christopher Kane   
8964    8467      St. Johnstone         64     711363  Diallang Jaiyesimi   
8965    8467      St. Johnstone         64     951994       Luke Jephcott   
8966    8467      St. Johnstone    

In [29]:
# Extracting player data and updating the DataFrame
updated_df = pd.DataFrame(columns=df.columns)  # Create an empty DataFrame with the same columns

for index, row in df.iterrows():
    player_id = row['Player_ID']

    # Building the URL for player data
    player_data_url = f"https://www.fotmob.com/api/playerData?id={player_id}"
    
    try:
        # Making the request for player data
        response = requests.get(player_data_url, headers={'User-Agent': 'Chrome/115.0.0.0'})
        
        # Checking the status code
        if response.status_code == 200:
            # Trying to parse JSON content
            player_data = response.json()

            # Check if "statSeasons" is not None in the JSON
            if player_data.get("statSeasons") is not None:
                # Check if there's a season with "seasonName" equal to "2023/2024"
                season_2023_2024 = next((season for season in player_data["statSeasons"] if season.get("seasonName") == "2023/2024"), None)

                if season_2023_2024:
                    # Check if the tournamentId for the league is present in the "tournaments" section
                    league_id = row['League_ID']
                    tournament_found = any(tournament.get("tournamentId") == league_id for tournament in season_2023_2024.get("tournaments", []))

                    if tournament_found:
                        # Append the player to the updated DataFrame
                        updated_df = updated_df.append(row, ignore_index=True)
                    else:
                        print(f"Player ID {player_id} doesn't have data for league ID {league_id} in the 2023/2024 season.")
                else:
                    print(f"Player ID {player_id} doesn't have data for the 2023/2024 season.")
            else:
                print(f"Player ID {player_id} doesn't have statistical data.")
        else:
            print(f"Failed to fetch data for player ID {player_id}. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        # Handle request exception for player data
        print(f"Failed to make request for player ID {player_id}. Exception: {e}")

# Displaying the updated DataFrame
print(updated_df)

Player ID 95336 doesn't have data for league ID 47 in the 2023/2024 season.


C:\Users\kikon\AppData\Local\Temp\ipykernel_26912\3887175065.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = updated_df.append(row, ignore_index=True)


Player ID 795539 doesn't have data for the 2023/2024 season.
Player ID 862993 doesn't have data for the 2023/2024 season.
Player ID 1290962 doesn't have data for the 2023/2024 season.
Player ID 773991 doesn't have data for league ID 47 in the 2023/2024 season.
Player ID 982692 doesn't have data for the 2023/2024 season.
Player ID 1013046 doesn't have statistical data.
Player ID 160627 doesn't have data for league ID 47 in the 2023/2024 season.
Player ID 1199709 doesn't have data for league ID 47 in the 2023/2024 season.
Player ID 36373 doesn't have data for the 2023/2024 season.
Player ID 533127 doesn't have data for the 2023/2024 season.
Player ID 1414691 doesn't have data for the 2023/2024 season.
Player ID 1333449 doesn't have data for league ID 47 in the 2023/2024 season.
Player ID 1207956 doesn't have data for league ID 47 in the 2023/2024 season.
Player ID 169756 doesn't have data for the 2023/2024 season.
Player ID 1068920 doesn't have data for the 2023/2024 season.
Player ID 11

In [33]:
# Extracting player stats and building a list of dictionaries
player_stats_list = []

for index, row in updated_df.iterrows():
    player_id = row['Player_ID']
    league_id = row['League_ID']
    
    # Building the URL
    url = f"https://www.fotmob.com/api/playerStats?playerId={player_id}&seasonId=2023%2F2024-{league_id}"
    
    # Making the request
    response = requests.get(url, headers={'User-Agent': 'Chrome/115.0.0.0'})
    
    # Checking the status code
    if response.status_code == 200:
        player_stats = response.json()
        
        # Checking if "statsSection" is present in the JSON
        if "statsSection" in player_stats:
            # Processing the player stats
            data = {"Player_ID": player_id, "Name": row["Name"]}
            
            for section in player_stats["statsSection"]["items"]:
                for item in section["items"]:
                    column_name = f"{section['title']}_{item['title']}"
                    data[column_name] = item["statValue"]
                
            # Appending the player stats to the list
            player_stats_list.append(data)
        else:
            print(f"Stats not found for player ID {player_id}")
       
    else:
        print(f"Failed to fetch data for player ID {player_id}. Status code: {response.status_code}")

# Creating a list of DataFrames
dfs = [pd.DataFrame([data]) for data in player_stats_list]

# Concatenating the list of DataFrames
final_df = pd.concat(dfs, ignore_index=True)

# Displaying the final DataFrame
print(final_df)

Stats not found for player ID 163670
Stats not found for player ID 1250658
Stats not found for player ID 339992
Stats not found for player ID 748382
Stats not found for player ID 844477
Stats not found for player ID 609755
Stats not found for player ID 1113790
Stats not found for player ID 1231075
Stats not found for player ID 1324871
Stats not found for player ID 303059
Stats not found for player ID 1088066
Stats not found for player ID 259306
Stats not found for player ID 308932
Stats not found for player ID 155913
Stats not found for player ID 891855
Stats not found for player ID 846381
Stats not found for player ID 426880
Stats not found for player ID 1203658
Stats not found for player ID 759566
Stats not found for player ID 363357
Stats not found for player ID 1190867
Stats not found for player ID 1458868
Stats not found for player ID 360918
Stats not found for player ID 562726
Stats not found for player ID 109897
Stats not found for player ID 1192854
Stats not found for player ID

In [34]:
final_df.to_excel("FOTMOB_Player_Stats.xlsx", index=False)